In [1]:
# face recognition

In [2]:
pip install face_recognition 

  Using cached face_recognition-1.3.0-py2.py3-none-any.whl (15 kB)
  Using cached face_recognition_models-0.3.0-py2.py3-none-any.whl
  Using cached dlib-19.24.2-cp39-cp39-win_amd64.whl
Note: you may need to restart the kernel to use updated packages.


DEPRECATION: tensorflow-examples ef9fcf969bc2f6a848ad69cfcd5f1dcd9474a1fb- has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of tensorflow-examples or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import cv2
import mediapipe as mp
import face_recognition

ModuleNotFoundError: No module named 'face_recognition'

In [5]:
mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils
face_detection = mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.5)

cap = cv2.VideoCapture(0)

frame_count = 0
face_encoding = None

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Face detection
    results = face_detection.process(img_rgb)

    if results.detections:
        for detection in results.detections:
            mp_drawing.draw_detection(frame, detection)

        if frame_count == 5:
            face_locations = face_recognition.face_locations(img_rgb)
            if face_locations:
                face_encoding = face_recognition.face_encodings(img_rgb, known_face_locations=face_locations)[0]

    cv2.imshow('Frame', frame)

    if cv2.waitKey(15) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

face_detection.close()

if face_encoding is not None:
    print("Face encoding from the 5th frame:", face_encoding)


KeyboardInterrupt: 

In [ ]:
mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils
face_detection = mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.5)

cap = cv2.VideoCapture(0)

frame_count = 0
fourth_frame_encoding = None

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    results = face_detection.process(img_rgb)

    if results.detections:
        for detection in results.detections:
            bboxC = detection.location_data.relative_bounding_box
            ih, iw, _ = frame.shape
            x, y, w, h = int(bboxC.xmin * iw), int(bboxC.ymin * ih), int(bboxC.width * iw), int(bboxC.height * ih)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

            # Extract face encoding and compare with the 4th frame (think that 4th frame is target face)
            face_locations = [(y, x+w, y+h, x)]
            face_encodings = face_recognition.face_encodings(img_rgb, known_face_locations=face_locations)

            if face_encodings:
                if frame_count == 4:
                    fourth_frame_encoding = face_encodings[0]
                elif fourth_frame_encoding is not None:
                    matches = face_recognition.compare_faces([fourth_frame_encoding], face_encodings[0], tolerance=0.6)
                    label = "Match" if matches[0] else "No Match"
                    cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

    cv2.imshow('Frame', frame)

    if cv2.waitKey(15) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
face_detection.close()
